In [ ]:
from utils.custom_dataset import YOLO6ChannelDataset

train_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/train/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/train/right",
    label_dir="D:/intezet/Bogi/Yolo/data/labels/train"
)

val_dataset = YOLO6ChannelDataset(
    left_img_dir="D:/intezet/Bogi/Yolo/data/images/val/left",
    right_img_dir="D:/intezet/Bogi/Yolo/data/images/val/right",
    label_dir="D:/intezet/Bogi/Yolo/data/labels/val"
)

In [14]:
import torch
from torch.utils.data import DataLoader
from ultralytics.nn.tasks import DetectionModel
from types import SimpleNamespace
from tqdm import tqdm
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 50
batch_size = 8
save_path = "yolov8_custom6.pt"


# --- Collate Function ---
def collate_fn(batch):
    imgs, targets = zip(*batch)
    imgs = torch.stack(imgs, dim=0)
    targets_with_id = []
    for i, t in enumerate(targets):
        if t.numel() == 0:
            continue
        img_idx = torch.full((t.shape[0], 1), i, dtype=t.dtype)
        targets_with_id.append(torch.cat([img_idx, t], dim=1))
    if len(targets_with_id) == 0:
        targets_combined = torch.zeros((0, 6))
    else:
        targets_combined = torch.cat(targets_with_id, dim=0)
    # print(f"targets_combined: {targets_combined}")
    return imgs, targets_combined


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                          collate_fn=collate_fn, num_workers=0)

val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        collate_fn=collate_fn, num_workers=0)


# --- Model ---
model = DetectionModel(cfg='ultralytics/cfg/models/v8/yolov8n.yaml', ch=6, nc=6).to(device)
model.args = SimpleNamespace(box=7.5, cls=0.5, dfl=1.5, reg_max=15)
model.criterion = model.init_criterion()
model.to(device)
# print(f"Expected output channels: {(model.args.reg_max + 1) * 4 + model.nc}")

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# --- Training Loop ---
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}")
    
    for imgs, targets in pbar:
        # print(f"imgs.shape: {imgs.shape}")
        imgs = imgs.to(device)
        targets = targets.to(device)

        if targets.shape[0] == 0:
            continue

        optimizer.zero_grad()
        preds = model(imgs)

        batch = {
            "img": imgs,
            "batch_idx": targets[:, 0],
            "cls": targets[:, 1],
            "bboxes": targets[:, 2:]
        }
        
        loss, loss_items = model.loss(batch, preds)  # loss is scalar, loss_items is [box, cls, dfl]
        loss = loss.sum()

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"📘 Epoch {epoch+1} - Train Loss: {avg_loss:.4f}")

    # --- Validation ---
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for imgs, targets in val_loader:
            imgs = imgs.to(device)
            targets = targets.to(device)

            if targets.shape[0] == 0:
                continue

            preds = model(imgs)
            batch = {
                "img": imgs,
                "batch_idx": targets[:, 0],
                "cls": targets[:, 1],
                "bboxes": targets[:, 2:]
            }
            
            loss_out, loss_out_items = model.loss(batch, preds)  # loss is scalar, loss_items is [box, cls, dfl]
            loss_out = loss_out.sum()
            val_loss += loss_out.item()

    avg_val_loss = val_loss / len(val_loader)
    print(f"🧪 Epoch {epoch+1} - Val Loss: {avg_val_loss:.4f}")

    # --- Save ---
    if (epoch + 1) % 5 == 0 or (epoch + 1) == epochs:
        torch.save(model.state_dict(), save_path)
        print(f"✅ Model saved to: {save_path}")


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       896  ultralytics.nn.modules.conv.Conv             [6, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                


  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  8                  -1  1    460288  ultralytics.nn.modules.block.C2f             [256, 256, 1, True]           
  9                  -1  1    164608  ultralytics.nn.modules.block.SPPF            [256, 256, 5]                 
 10                  -1  1         0  torch.nn.modules.upsampling.Upsample         [None

Epoch 1/50: 100%|██████████| 17/17 [01:40<00:00,  5.92s/it, loss=379]


📘 Epoch 1 - Train Loss: 536.0597
🧪 Epoch 1 - Val Loss: 426.5753


Epoch 2/50: 100%|██████████| 17/17 [01:38<00:00,  5.78s/it, loss=362]


...

📘 Epoch 48 - Train Loss: 158.6298
🧪 Epoch 48 - Val Loss: 142.9610


Epoch 49/50: 100%|██████████| 17/17 [01:37<00:00,  5.76s/it, loss=119]


📘 Epoch 49 - Train Loss: 154.8891
🧪 Epoch 49 - Val Loss: 138.5046


Epoch 50/50: 100%|██████████| 17/17 [01:36<00:00,  5.65s/it, loss=116]


📘 Epoch 50 - Train Loss: 151.4806
🧪 Epoch 50 - Val Loss: 135.6620
✅ Model saved to: yolov8_custom6.pt
